# Economics Tie-In

It's considered general knowledge that the worse state the economy/country is in, the better the entertainment industry does. But is this true? Can we see any sort of relationship between the state of the film industry and the standings of average means in the US?

To find out, we will be using two datasets (one for average wages by metric, and one for unemployment rates by metric) to create one economic DataFrame (DF) from which we can draw our comparrisons.

In [ ]:
# Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt

# Data

---

The following datasets are courtesy of __[Kaggle](https://www.kaggle.com/)__.

### __['...CCI_OECD.csv'](https://www.kaggle.com/datasets/iqbalsyahakbar/cci-oecd)__

*renamed from `DP_LIVE_16112023095843236.csv`*

Per the Organisation for Economic Co-operation and Development (OECD);

* The Consumer Confidence Indicator (CCI) is an indication of developments for future households' consumption and saving based on expected financial situation, sentiment regarding the general economic situation, employment status, and capacity for savings
* An indicator above `100` indicates an optimistic outlook and a greater likliehood to spend money over cautious saving
* An indicator below `100` indicates a pessimistic outlook and both a higher likeliehood to save money and a lower tendency to consume

### __['...US_inflation_rates.csv'](https://www.kaggle.com/datasets/pavankrishnanarne/us-inflation-dataset-1947-present)__

Per the dataset description;

* The Consumer Price Index (CPI) is a critical economic indicator for measuring the purchasing power of money over time, measuring the average change over time in the prices paid by urban consumers for goods and services
* The CPI is the value at the end of the respective month

### __['...public_debt.csv'](https://www.kaggle.com/datasets/pavankrishnanarne/us-public-debt-quarterly-data-1996-present)__

Per the dataset description;

* Public debt, or government debt, represents the total outstanding bonds and other securities of a country's central government
* The provided `value` represents the quarterly US public debt in billions of US Dollars (USD)

---

The following datasets are courtesy of the __[Economic Policy Institute’s (EPI) State of Working America Data Library](https://www.epi.org/data/)__.

### __['...Medianaverage hourly wages.csv'](https://www.epi.org/data/#?subject=wage-avg)__

Per EPI description;

* Wages adjusted to 2023 dollars by the Consumer Price Index for All Urban Consumers Research Series
* `Median Wage` is the hourly wage in the middle of the wage distribution
* `Average Wage` is the calculated mean of all wages

### __['...Unemployment.csv'](https://www.epi.org/data/#?subject=unemp)__

Per EPI description;

* Unemployment is the share of the labor force wihout a job
* Monthly percentages calculated as a rolling 12-month average (mean)

In [ ]:
# Reading in data
df_hourly_wages = pd.read_csv("./Resources/EPI Data Library - Medianaverage hourly wages.csv")
df_unemp = pd.read_csv("./Resources/EPI Data Library - Unemployment.csv")
df_cci = pd.read_csv("./Resources/CCI_OECD.csv")
df_inflation = pd.read_csv("./Resources/US_inflation_rates.csv")
df_public_debt = pd.read_csv("./Resources/public_debt.csv")

# Defining functions

Since each dataset will need similar preprocessing, the following functions will be used to help streamline the flow and code.

### Universal functions

Applicable to all datasets

#### Copying datasets

Creating a working copy of a given dataset to preserve the original DF with unneeded features dropped

#### Renaming needed features

Renaming selected features for a given dataset

#### Rolling mean and mean percent change

Calculating the rolling 12-month mean and the rolling 12-month percent change for a given feature

In [ ]:
# Defining a function to copy a dataset with only the needed features
def copy_df(df, features_to_keep):
    df_copy = df[features_to_keep].copy()
    return df_copy

# Defining a function to rename needed features
def rename_features(df, feature1, feature1new, feature2, feature2new):
    df.rename(columns={
        feature1: feature1new,
        feature2: feature2new
    }, inplace=True)
    return df

# Defining a function to calculate the rolling 12-month means and percent changes
# for a given feature
def rolling_calcs(df, feature, feature_mean, feature_pct_chng):
    df[feature_mean] = df[feature].rolling(window=12).mean()
    df[feature_pct_chng] = df[feature_mean].pct_change(periods=12) * 100
    return df

### Situational functions

Applicable to select datasets

#### Datetime indexing

Converting the feature containing the raw datetime information into a suitable datetime index

*Cannot be used on `Hourly Wages` or `Unemployment` datasets*

#### Forward filling

Resampling by using quarterly or annual data to fill in the missing months of a datetime index

*For `Public Debt` and `Hourly Wages` datasets*

#### Removing '$'

Removing the `'$'` from a given feature and converting the remaining `object` dtype to `float`

*Sepcifically for `Hourly Wages` dataset*

#### Removing '%'

Removing the `'%'` from a given feature and converting the remaining `object` dtype to `float`

*Specigically for `Unemployment` dataset*

In [ ]:
# Defining a function to set a `Date` feature as a datetime index
def datetime_index(df, datetime_feature):
    df[datetime_feature] = pd.to_datetime(df[datetime_feature])
    df.set_index(datetime_feature, inplace=True)
    df.sort_index(inplace=True)
    return df

# Defining a function to resample missing months by forward filling the datetime index
def forward_fill(df):
    df = df.resample('MS').ffill()
    return df

# Defining a function to remove '$' and convert data to `float`
def convert_dollars(feature):
    if isinstance(feature, str):
        return float(feature.strip('$'))
    return feature

# Defining a function to apply `convert_dollars`
def apply_dollars(df, feature):
    df[feature] = df[feature].apply(convert_dollars)
    return df

# Defining a function to remove '%' and convert data `float`
def convert_percentage(feature):
    return float(feature.strip('%'))

# Defining a function to apply `convert_percentage`
def apply_percentage(df, feature):
    df[feature] = df[feature].apply(convert_percentage)
    return df

# CCI

#### Preprocessing of the `CCI_OECD.csv` dataset

This dataset came with internaitonal records and unneeded features, so only records for US CCI will be retained. Once those records have been selected, the resulting DF will need to be prepared for concatenation with the remainined economic datasets. To do this, the `TIME` feature will be converted to datetime and set as the index.

In [ ]:
# Viewing `df_cci`
df_cci.head()

#### Beginning of limited EDA

In [ ]:
# Beginning EDA on `df_cci`
df_cci.describe()

In [ ]:
# Continuing EDA
df_cci.shape

In [ ]:
# Continuing EDA
df_cci.dtypes

In [ ]:
# Continuing EDA
df_cci['LOCATION'].unique()

#### Selecting only domestic data

In [ ]:
# Copying domestic data from `df_cci` to `df_cci_us` and removing unneeded features
df_cci_us = df_cci.loc[df_cci['LOCATION'] == 'USA'].copy()

#### Applying defined functions

In [ ]:
# Copying `df_cci_us` and dropping unneeded features
df_cci_form = copy_df(df_cci_us, ['TIME', 'Value'])

# Renamining retained features
df_cci_form = rename_features(
    df_cci_form, 'TIME', 'Date', 'Value', 'CCI Value'
)

# Converting `Date` to a datetime index
df_cci_form = datetime_index(df_cci_form, 'Date')

# Calculating rolling 12-month means and percent change in means
df_cci_form = rolling_calcs(
    df_cci_form, 'CCI Value', 'CCI Rolling Mean', 'CCI Rolling Percent Change'
)

# Confirming `df_cci_form` ready to concatenate
display(df_cci_form.head())
display(df_cci_form.tail())

# Inflation

#### Preprocessing of the `US_inflation_rates.csv` dataset

Seeing as the dataset came with only the needed features, little will be needed to prepare the DF for concatenation with the other economic datasets. `date` will be converted to datetime and set as the index.

In [ ]:
# Viewing `df_inflation`
df_inflation.head()

#### Beginning on limited EDA

In [ ]:
# Beginning of EDA on `df_inflation`
df_inflation.describe()

In [ ]:
# Continuing EDA
df_inflation.shape

In [ ]:
# Continuing EDA
df_inflation.dtypes

#### Applying defined functions

In [ ]:
df_inflation.columns

In [ ]:
# Copying `df_inflation` and dropping unneeded features
df_inflation_form = copy_df(df_inflation, ['date', 'value'])

# Renamining retained features
df_inflation_form = rename_features(
    df_inflation_form, 'date', 'Date', 'value', 'CPI Value'
)

# Converting `Date` to a datetime index
df_inflation_form = datetime_index(df_inflation_form, 'Date')

# Calculating rolling 12-month means and percent change in means
df_inflation_form = rolling_calcs(
    df_inflation_form,
    'CPI Value',
    'CPI Rolling Mean',
    'CPI Rolling Percent Change'
)

# Confirming `df_inflation_form` ready to concatenate
display(df_inflation_form.head())
display(df_inflation_form.tail())

# Public Debt

#### Preprocessing of the `public_debt.csv` dataset

Since this datset came as quarterly values, both converting `date` to datetime and the index will be necessary to resample for the missing months. Once the values are forward filled, the DF will be ready to concatenate with the other economic datasets.

In [ ]:
# Viewing `df_public_debt`
df_public_debt.head()

#### Beginning of limited EDA

In [ ]:
# Beginning of EDA on `df_public_debt`
df_public_debt.describe()

In [ ]:
# Continuing EDA
df_public_debt.shape

In [ ]:
# Continuing EDA
df_public_debt.dtypes

#### Applying defined functions

In [ ]:
# Copying `df_public_debt` and dropping unneeded features
df_public_debt_form = copy_df(df_public_debt, ['date', 'value'])

# Renamining retained features
df_public_debt_form = rename_features(
    df_public_debt_form, 'date', 'Date', 'value', 'Public Debt (bil USD)'
)

# Converting `Date` to a datetime index
df_public_debt_form = datetime_index(df_public_debt_form, 'Date')

# Forward filling missing months
df_public_debt_form = forward_fill(df_public_debt_form)

# Calculating rolling 12-month means and percent change in means
df_public_debt_form = rolling_calcs(
    df_public_debt_form,
    'Public Debt (bil USD)',
    'Public Debt (bil USD) Rolling Mean',
    'Public Debt Rolling Percent Change'
)

# Confirming `df_public_debt_form` ready to concatenate
display(df_public_debt_form.head())
display(df_public_debt_form.tail())

## Hourly Wages

#### Preprocessing of the `Medianaverage hourly wages.csv` dataset

This dataset came with unneeded features that will need to be dropped, as well as the needed features will need to be converted to `float`. Additionally, as an annual measures of mean and median wages, the `Date` feature will need to be converted to datetime and set to the index in order to resample for the missing months. Once the values are forward filled, the DF will be ready to concatenate with with other economic datasets.

In [ ]:
# Viewing `df_hourly_wages`
df_hourly_wages.head()

#### Beginning of limited EDA

In [ ]:
# Beginning EDA on `df_hourly_wages`
df_hourly_wages.describe()

In [ ]:
# Continuing EDA
df_hourly_wages.shape

In [ ]:
# Continuing EDA
df_hourly_wages.dtypes

#### Applying defined functions (first pass)

*Given the nature of the* `Date` *feature in this dataset, the datetime indexing will need to be handled outside of the defined functions*

In [ ]:
# Copying `df_hourly_wages` and dropping unneeded features
df_wages_form = copy_df(df_hourly_wages, ['Date', 'Median', 'Average'])

# Renamining retained features
df_wages_form = rename_features(
    df_wages_form,
    'Median',
    'Median Annual Hourly Wage (USD)',
    'Average',
    'Mean Annual Hourly Wage (USD)'
)

#### Datetime indexing

In [ ]:
# Converting `Date` to datetime
df_wages_form['Date'] = pd.to_datetime(df_wages_form['Date'], format='%Y')

# Setting `Date` as index
df_wages_form.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_wages_form.sort_index(inplace=True)

#### Applying defined functions (second pass)

In [ ]:
# Forward filling missing months
df_wages_form = forward_fill(df_wages_form)

# Applying `apply_dollars` to `Median Annual...`
df_wages_form = apply_dollars(df_wages_form, 'Median Annual Hourly Wage (USD)')

# Applying `apply_dollars` to `Mean Annual...`
df_wages_form = apply_dollars(df_wages_form, 'Mean Annual Hourly Wage (USD)')

# Calculating rolling 12-month means and percent change in means
df_wages_form = rolling_calcs(
    df_wages_form,
    'Median Annual Hourly Wage (USD)',
    'Median Annual Hourly Wage (USD) Rolling Mean',
    'Median Annual Hourly Wage Rolling Percent Change'
)

# Calculating rolling 12-month means and percent change in means
df_wages_form = rolling_calcs(
    df_wages_form,
    'Mean Annual Hourly Wage (USD)',
    'Mean Annual Hourly Wage (USD) Rolling Mean',
    'Mean Annual Hourly Wage Rolling Percent Change'
)

# Confirming `df_public_debt_form` ready to concatenate
display(df_wages_form.head())
display(df_wages_form.tail())

## Unemployment

#### Preprocessing of the `Unemployment.csv` dataset

This dataset came with unneeded features that will need to be dropped, as well as the needed features will need to be converted to `float`. Additionally, the `Date` feature will need to be converted to datetime and set to the index in preparation for concatenation with the other economic datasets.

In [ ]:
# Viewing `df_unemp`
df_unemp.head()

#### Beginning of limited EDA

In [ ]:
# Beginning EDA on `df_unemp`
df_unemp.describe()

In [ ]:
# Continuing EDA
df_unemp.shape

In [ ]:
# Continuing EDA
df_unemp.dtypes

#### Applying defined functions (first pass)

*Given the nature of the* `Date` *feature in this dataset, the datetime indexing will need to be handled outside of the defined functions*

In [ ]:
# Copying `df_unemp` and dropping unneeded features
df_unemp_form = copy_df(df_unemp, ['Date', 'All'])

#### Renaming needed feature

*This dataset only needed one feature,* `All`*, to be renmaned, this the* `rename_features` *defined function is not applicable*

In [ ]:
# Renaming the reatined feature
df_unemp_form.rename(columns={'All': 'Unemployment Rate (%)'}, inplace=True)

#### Datetime indexing

In [ ]:
# The `Date` feature will need to be engineered into a workable datetime feature

# Creating a dictionary of Months
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Mapping integer month values to `Date Month`
df_unemp_form['Date Month'] = df_unemp_form['Date'].str.slice(0,3).map(month_map)

# Slicing `Date Year`
df_unemp_form['Date Year'] = df_unemp_form['Date'].str.slice(4,8)

# Converting `Date` to datetime using `Date Month` and `Date Year`
df_unemp_form['Date'] = pd.to_datetime({
    'year': df_unemp_form['Date Year'],
    'month': df_unemp_form['Date Month'],
    'day': 1
})

# Dropping engineered features `Date Month` and `Date Year`
df_unemp_form.drop(columns=['Date Month', 'Date Year'], inplace=True)

# Setting `Date` as index
df_unemp_form.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_unemp_form.sort_index(inplace=True)

#### Applying defined functions (second pass)

In [ ]:
# Applying `apply_percentage` to `Unemployment Rate (%)`
df_unemp_form = apply_percentage(df_unemp_form, 'Unemployment Rate (%)')

# Calculating rolling 12-month means and percent change in means
df_unemp_form = rolling_calcs(
    df_unemp_form,
    'Unemployment Rate (%)',
    'Unemployment Rate (%) Rolling Mean',
    'Unemployment Rate Rolling Percent Change',
)

# Confirming `df_unemp_form` ready to concatenate
display(df_unemp_form.head())
display(df_unemp_form.tail())

## Combined Economics

#### Preprocessing of the `df_economics` DF

With all datasets set to a monthly datetime index, the relevent features of all can be combined into one DF, and any NaN records can be dropped.

In [ ]:
df_cci_form.shape, df_inflation_form.shape, df_public_debt_form.shape, df_wages_form.shape, df_unemp_form.shape

In [ ]:
# Concatenating the five economic datasets into `df_economics`
df_economics = pd.concat(
    [
        df_cci_form,
        df_inflation_form,
        df_public_debt_form,
        df_wages_form,
        df_unemp_form
    ], axis=1, join='outer'
)

#### Handling `NaN` rows

In [ ]:
# Confirming total records
df_economics.shape

In [ ]:
df_economics.head(36)

In [ ]:
df_economics.tail(36)

In [ ]:
# Checking total `NaN` rows
df_economics.isna().count()

In [ ]:
# Dropping `NaN` rows
df_economics.dropna(inplace=True)

# Confirming remaining records
df_economics.shape

In [ ]:
# Confirming final economic DF
display(df_economics.head())
display(df_economics.tail())

## Proof of Concept

#### Engineering the economic target value

As a feature will need to be engineered for the final modelling, the following cells will be used to test the scaling and thresholds for classification as a proof of concept prior to merging the economic and movie datasets. A working DF will be used so that `df_economics` may be retained and untouched for modeling as a complete dataset.

For the purposes of this feature engineering, we will be looking to classify `Economic State` by the following definitions;

* Boom - Strong economic growth, low unemployment, high consumer confidence, and stable or increasing wages
* Expansion - Economic growth is positive, but not as robust as during a Boom
* Recession - Economic growth is negative, rising unemployment, and decreasing consumer confidence
* Contraction - Economic growth is negative, but not as severe as a Recession
* Recovery - Economic growth is starting to pick up after a Recession or Contraction
* Stagnation - Economic grown is very low or unmoving, with high unemployment and low consumer confidence

Given the data present in the `df_economics` dataset, we can interpret the values and trends in rolling windows to make this classification possible.

* **CCI Value**: Values over `100` indicate stronger consumer confidence, and values below `100` indicate stronger consumer caution. Trends will help distinguish between `Boom`, `Recession`, `Stagnation`, and the states between.
* **CPI Value**: Represents inflation. Increasing CPI might indicate inflationary pressures seen during Boom periods, while stable or decreasing CPI could indicate lower economic activity associated with Recession or Contraction.
* **Quarterly Public Debt**: Higher public debt might indicate economic stress and government borrowing to stimulate the economy, often seen in Recession or Contraction. Lower or stable public debt might be seen in Boom or Expansion periods.
* **Annual Hourly Median Wage**: Increasing median wages are associated with Boom or Expansion, while stagnant or decreasing median wages might be seen in Recession or Contraction.
* **Annual Hourly Mean Wage**: Similar to median wages, increasing mean wages indicate Boom or Expansion, while stagnant or decreasing mean wages might be seen in Recession or Contraction.
* **Monthly Unemployment Rate**: Lower unemployment rates are associated with Boom or Expansion, while higher unemployment rates are seen in Recession or Contraction.

By using a rolling 12-month window, we can illustrate the trends for each feature in any given month, and create the feature `Economic State`.

#### Continued EDA

In [ ]:
# Creating the working DF `df_eco_test` as a copy of `df_economics`
df_eco_test = df_economics.copy()

# Viewing `df_eco_test`
df_eco_test.head()

In [ ]:
# Continuing EDA
df_eco_test.describe()

#### Classifying `Economic State`

In [ ]:
df_eco_test.columns

In [ ]:
df_eco_test[[
    'CCI Rolling Percent Change',
    'CPI Rolling Percent Change',
    'Public Debt Rolling Percent Change',
    'Median Annual Hourly Wage Rolling Percent Change',
    'Mean Annual Hourly Wage Rolling Percent Change',
    'Unemployment Rate Rolling Percent Change'
]].describe()

In [ ]:
# Assigning `Economic State` flags per feature

# Declaring a list of features to assign flags
features_to_flag = [
    'CCI Rolling Percent Change',
    'CPI Rolling Percent Change',
    'Public Debt Rolling Percent Change',
    'Median Annual Hourly Wage Rolling Percent Change',
    'Mean Annual Hourly Wage Rolling Percent Change',
    'Unemployment Rate Rolling Percent Change'
]

# Defining a function to flag each feature in `features_to_flag`
def flag_economics(df, features_to_flag):
    for col in df[features_to_flag].columns:
        new_col = str(col) + ' Flag'
        mean = df[col].mean()
        stnd = df[col].std()
        str_pos = mean + (1.5 * stnd)
        mod_pos = mean + (0.5 * stnd)
        mod_neg = mean - (0.5 * stnd)
        str_neg = mean - (1.5 * stnd)
        df[new_col] = 'neutral'
        df.loc[df[col] >= str_pos, new_col] = 'strong positive'
        df.loc[(df[col] < str_pos) & (df[col] >= mod_pos), new_col] = 'moderate positive'
        df.loc[(df[col] < mod_pos) & (df[col] > mod_neg), new_col] = 'neutral'
        df.loc[(df[col] <= mod_neg) & (df[col] > str_neg), new_col] = 'moderate negative'
        df.loc[df[col] <= str_neg, new_col] = 'strong negative'
    return df

# Applying `flag_economics`
df_eco_test_2 = flag_economics(df_eco_test, features_to_flag)

In [ ]:
flag_cols = [
    'CCI Rolling Percent Change Flag',
    'CPI Rolling Percent Change Flag',
    'Public Debt Rolling Percent Change Flag',
    'Median Annual Hourly Wage Rolling Percent Change Flag',
    'Mean Annual Hourly Wage Rolling Percent Change Flag',
    'Unemployment Rate Rolling Percent Change Flag'
]

In [ ]:
df_eco_test_2[flag_cols].head()

In [ ]:
for col in flag_cols:
    display(df_eco_test_2[col].value_counts())

In [ ]:
# if:
#     Boom
#     CCI = Strong Positive
#     CPI = Moderate Positive OR Neutral
#     Public Debt = Neutral
#     Wages = Strong Positive
#     Unemployment = Strong Negative
# elif:
#     Expansion
#     CCI = Moderate Positive
#     CPI = Neutral
#     Public Debt = Neutral
#     Wages = Moderate Positive
#     Unemployment = Moderate Negative
# elif:
#     Recession
#     CCI = Strong Negative
#     CPI = Strong Negative
#     Public Debt = Strong Positive
#     Wages = Moderate OR Strong Negative
#     Unemployment = Strong Positive
# elif:
#     Contraction
#     CCI = Moderate Negative
#     CPI = Moderate OR Strong Negative
#     Public Debt = Strong Positive
#     Wages = Moderate OR Strong Negative
#     Unemployment = Strong Positive
# elif:
#     Recovery
#     CCI = Neutral OR Moderate Positive
#     CPI = Neutral
#     Public Debt = Neutral
#     Wages = Moderate Positive
#     Unemployment = Moderate Negative
# elif:
#     Stagnation
#     CCI = Neutral
#     CPI = Neutral
#     Public Debt = Neutral
#     Wages = Neutral
#     Unemployment = Strong Negative
# else:
#     Unknown

# Boom                                      Tot: 0
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'strong positive') &
#     (df_eco_test_2[flag_cols[1]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[1]] == 'neutral') &
#     (df_eco_test_2[flag_cols[2]] == 'neutral') &
#     (df_eco_test_2[flag_cols[3]] == 'strong positive') &
#     (df_eco_test_2[flag_cols[4]] == 'strong positive') &
#     (df_eco_test_2[flag_cols[5]] == 'strong negative')
# ].shape

# Expansion                                 Tot: 0
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[1]] == 'neutral') &
#     (df_eco_test_2[flag_cols[2]] == 'neutral') &
#     (df_eco_test_2[flag_cols[3]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[4]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[5]] == 'moderate negative')
# ].shape

# Recession                                 Tot: 14
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[1]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[2]] == 'strong positive') &
#     (df_eco_test_2[flag_cols[3]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[3]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[4]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[4]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[5]] == 'strong positive')
# ].shape

# Contraction                               Tot: 28
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'moderate negative') &
#     (df_eco_test_2[flag_cols[1]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[1]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[2]] == 'strong positive') &
#     (df_eco_test_2[flag_cols[3]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[3]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[4]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[4]] == 'strong negative') &
#     (df_eco_test_2[flag_cols[5]] == 'strong positive')
# ].shape

# Recovery                                  Tot: 245
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'neutral') |
#     (df_eco_test_2[flag_cols[0]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[1]] == 'neutral') &
#     (df_eco_test_2[flag_cols[2]] == 'neutral') &
#     (df_eco_test_2[flag_cols[3]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[4]] == 'moderate positive') &
#     (df_eco_test_2[flag_cols[5]] == 'moderate negative')
# ].shape

# Stagnation                                Tot: 0
# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'neutral') &
#     (df_eco_test_2[flag_cols[1]] == 'neutral') &
#     (df_eco_test_2[flag_cols[2]] == 'neutral') &
#     (df_eco_test_2[flag_cols[3]] == 'neutral') &
#     (df_eco_test_2[flag_cols[4]] == 'neutral') &
#     (df_eco_test_2[flag_cols[5]] == 'strong negative')
# ].shape

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[0]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[0]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[0]] == 'neutral') |
#     (df_eco_test_2[flag_cols[0]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[0]] == 'strong negative')
# ]

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[1]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[1]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[1]] == 'neutral') |
#     (df_eco_test_2[flag_cols[1]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[1]] == 'strong negative')
# ]

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[2]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[2]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[2]] == 'neutral') |
#     (df_eco_test_2[flag_cols[2]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[2]] == 'strong negative')
# ]

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[3]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[3]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[3]] == 'neutral') |
#     (df_eco_test_2[flag_cols[3]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[3]] == 'strong negative')
# ]

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[4]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[4]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[4]] == 'neutral') |
#     (df_eco_test_2[flag_cols[4]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[4]] == 'strong negative')
# ]

# df_eco_test_2[flag_cols].loc[
#     (df_eco_test_2[flag_cols[5]] == 'strong positive') |
#     (df_eco_test_2[flag_cols[5]] == 'moderate positive') |
#     (df_eco_test_2[flag_cols[5]] == 'neutral') |
#     (df_eco_test_2[flag_cols[5]] == 'moderate negative') |
#     (df_eco_test_2[flag_cols[5]] == 'strong negative')
# ]